In [6]:
pip install pygame


Note: you may need to restart the kernel to use updated packages.


In [7]:
import pygame
from IPython.display import Image

In [8]:
# FEATURES ADDED:
    #1 Emojis to the text (keys, intro text)
    #2 Color codes to the text
    #3 Added linebreaks across the code for UX purposes while playing
    #4 Added sounds: ambience, door opening, finishing the game
    #5 Added a replay function
    #6 Added two gifs
    #7 
    
# import pygame
# from IPython.display import Image

# Defining colour codes

BLUE = '\033[94m'
ENDC = '\033[0m'


# Defining items:

## Doors:

door_a = {
    "name": "door a",
    "type": "door",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}
door_d = {
    "name": "door d",
    "type": "door",
}

## Game Room: 

game_room = {
    "name": "game room",
    "type": "room",
}

couch = {
    "name": "couch",
    "type": "furniture",
}

key_a = {
    "name": "key for 🔑 door a",
    "type": "key",
    "target": door_a,
}

piano = {"name": "piano",
         "type": "furniture",
}
    
## Bedroom 1:

bedroom_1 = {
    "name": "bedroom 1",
    "type": "room",
}

queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}

key_b = {
    "name": "key for 🔑 door b",
    "type": "key",
    "target": door_b
}

## Bedroom 2:

bedroom_2 = {
    "name": "bedroom 2",
    "type": "room",
}

double_bed = {
    "name": "double bed",
    "type": "furniture",
}

key_c = {
    "name": "key for 🔑 door c",
    "type": "key",
    "target": door_c,
}

dresser = {
    "name": "dresser",
    "type": "furniture",
}

key_d = {
    "name": "key for 🔑 door d",
    "type": "key",
    "target": door_d,
}

## Living Room

living_room = {
    "name": "living room",
    "type": "room",
}

dining_table = {
    "name": "dining table",
    "type": "furniture",
}


## Target room: Outside

outside = {
    "name": "outside"
}

# Object relations 

object_relations = {
    "game room":[couch, piano, door_a],
    "piano": [key_a],
    "door a":[game_room, bedroom_1],
    "bedroom 1": [door_a, queen_bed, door_b, door_c],
    "queen bed": [key_b],
    "door b" : [bedroom_1, bedroom_2],
    "bedroom 2": [door_b, double_bed, dresser],
    "double bed": [key_c],
    "dresser": [key_d],
    "door c": [bedroom_1, living_room],
    "living room": [door_c, dining_table, door_d],
    "dining table": [],
    "door d": [living_room, outside],
    "outside": [door_d]
}


## Initialize the Pygame sound mixer
pygame.init()
pygame.mixer.init() 
  
## Loading the sounds 
sound_1 = pygame.mixer.Sound("door_sound.mp3")
sound_2 = pygame.mixer.Sound("haunted_ambience.mp3")
sound_3 = pygame.mixer.Sound("success.mp3")


# Define game state. Do not directly change this dict.
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This
# way you can replay the game multiple times.

INIT_GAME_STATE = {"current_room": game_room, "keys_collected": [], "target_room": outside}
game_state = INIT_GAME_STATE.copy()  
    

def linebreak():
    print("\n\n")
    
def door_sound():
    # Plays the sound of the door opening.
    return sound_1.play()

def haunted_sound():
    # Plays the sound of the haunted house ambience.
    return sound_2.play()

def success_sound():
    # Plays the sound of successfully finished.
    return sound_3.play()

def start_game():
    # Start the game.  
    
    haunted_sound()
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before.")
    print("You don't remember why you are here and what had happened before.")
    print("You feel some unknown danger 💥💥 is approaching and you must get out of the house, NOW! 🏃‍🏃‍🏃‍")
            
    linebreak()  
    print("You are now in Game Room")
    play_room(game_state["current_room"])
    

def play_again_prompt():
    # Prompt to replay the game once finished.
    
    play_again = input(f"{BLUE}Do you want to play again? (yes/no): {ENDC}").strip().lower()
    if play_again == "yes":
        game_state.clear()
        game_state.update(INIT_GAME_STATE)
        start_game()
    else:
        print("Thanks for playing! Goodbye.")
        return
    
def play_room(room):
    # Play a room. First check if the room being played is the target room.
    #If it is, the game will end with success. Otherwise, let player either
    #explore (list all items in this room) or examine an item found here.    
    
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        sound_2.stop()
        success_sound()
        text = "Congrats! 🎉 You escaped the room!🎈"
        display(Image(filename="outside.gif"))

        bold_text = "\033[1m" + text.upper()+ "\033[0m"
        print(bold_text)
        
        linebreak()
        play_again_prompt()

    else:       
        intended_action = input(f"{BLUE}What would you like to do? Type 'explore' or 'examine'?{ENDC}").lower().strip()

        if intended_action == "explore":
            explore_room(room)
            linebreak()
            play_room(room)

        elif intended_action == "examine":
            
            # OPTIONAL: these two lines can be added or not.
            # It gives a reminder of the items available to examine,
            # but it also makes the "explore" function less useful.
            # These were added by us, not in the original code.
            
            # items = [i["name"] for i in object_relations[room["name"]]]
            # print("You can examine " + ", ".join(items) + ".")
            
            examine_item(input(f"{BLUE}What would you like to examine?{ENDC}").lower().strip())
      
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
         
            play_room(room)
            linebreak()  
            
def explore_room(room):
    # Explore a room. List all items belonging to this room.
          
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items) + ".")
    

def get_next_room_of_door(door, current_room):
    # From object_relations, find the two rooms connected to the given door. Return the room that is not the current room.
     
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    # Examine an item which can be a door or furniture. First make sure the intended item belongs to the current room.
    # Then check if the item is a door. Tell player if the key hasn't been collected yet.
    # Otherwise, ask the player if they want to go to the next room. If the item is not a door, then check if it contains keys.
    # Collect the key if found and update the game state. At the end, play either the current or the next room
    # depending on the game state to keep playing.        
        
    current_room = game_state["current_room"]
    next_room = ""
    output = None

    for item in object_relations[current_room["name"]]:
        if item["name"] == item_name:
            output = "You examine " + item_name + ". "
            if item["type"] == "door":
                have_key = False
                for key in game_state["keys_collected"]:
                    if key["target"] == item:
                        have_key = True
                if have_key:
                    output += "\033[1m" + "You unlock it with a key you have. 🤩" "\033[0m"
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if item["name"] in object_relations and len(object_relations[item["name"]]) > 0:
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "\033[1m" + "You find " + item_found["name"] + "." "\033[0m"
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if output is None:
        print("The item you requested is not found in the current room.")
        
    if next_room and input(f"{BLUE}Do you want to go to the next room? Enter 'yes' or 'no': {ENDC}").lower().strip() == 'yes':    
        door_sound()
        display(Image(filename="scary.gif"))
        linebreak()
        print("You are now in " + next_room["name"] + ".")
        play_room(next_room)
        
    else:
        linebreak()
        play_room(current_room)    
        
        
           
        
           
        
        

FileNotFoundError: No file 'door_sound.mp3' found in working directory '/Users/jochemmeesters/Documents/Ironhack_DA/week1/day3/python-project/your-code'.

In [ ]:
game_state = INIT_GAME_STATE.copy()

start_game() 
